<a href="https://colab.research.google.com/github/nikeyes/GenAI-experiments/blob/main/query-my-documents/LLM_queries_with_urls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers
!pip install faiss-cpu
!pip install unstructured
!pip install chromadb
!pip install Cython
!pip install tiktoken
!pip install unstructured[local-inference]
# To avoid ImportError: cannot import name 'is_directory' from 'PIL._util' (/usr/local/lib/python3.10/dist-packages/PIL/_util.py)
!pip install --upgrade pillow

In [ ]:
import os
import requests
from configparser import ConfigParser
parser = ConfigParser()
_ = parser.read('config.ini')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = parser.get('HuggingFace','ACCESS_TOKEN')

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
urls = [
    "https://nikeyes.github.io/Principios-de-un-Open-Space-y-viajar-con-ni%C3%B1os-en-coche/",
    "https://nikeyes.github.io/cuando-cambiar-de-trabajo/"
]
loader = [UnstructuredURLLoader(urls=urls)]
#loader
doc = loader[0].load()


In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)).from_loaders(loader)

retriever=index.vectorstore.as_retriever()

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
docs = text_splitter.split_documents(doc)
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()#Create the vectorized db
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS
index = FAISS.from_documents(docs, embeddings)

retriever = index.as_retriever(search_kwargs={"k":3})

In [ ]:
query = "Explícame la ley de los 2 pies con niños."
#docs = index.similarity_search(query)
docs = index..search(query)
docs

In [ ]:
from langchain import HuggingFaceHub

llm=HuggingFaceHub(repo_id="declare-lab/flan-alpaca-large", model_kwargs={"temperature":0, "max_length":512})
#llm=HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.1, "max_length":512})
#llm=HuggingFaceHub(repo_id="mosaicml/mpt-7b", model_kwargs={"temperature":0.1, "max_length":512})


In [ ]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm, 
                                    chain_type="stuff", 
                                    retriever=retriever, 
                                    input_key="question",
                                    return_source_documents=True)

In [ ]:

#chain.run('Explícame la ley de los 2 pies con niños.')
chain('Explícame la ley de los 2 pies con niños.')

In [ ]:
ques = input('Your question: ')
chain.run(ques)